In [15]:
print("Hello WOrld")
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

Hello WOrld


In [2]:
import pyspark
import pyspark
from pyspark.sql import SparkSession
import os

## DEFINE SENSITIVE VARIABLES
NESSIE_URI = "http://nessie:19120/api/v1"
MINIO_ACCESS_KEY = "admin"
MINIO_SECRET_KEY = "password"



conf = (
    pyspark.SparkConf()
        .setAppName('app_name')
  		#packages
        .set('spark.jars.packages', 'org.apache.iceberg:iceberg-spark-runtime-3.3_2.12:1.3.1,org.projectnessie.nessie-integrations:nessie-spark-extensions-3.3_2.12:0.67.0,software.amazon.awssdk:bundle:2.17.178,software.amazon.awssdk:url-connection-client:2.17.178')
  		#SQL Extensions
        .set('spark.sql.extensions', 'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions') #org.projectnessie.spark.extensions.NessieSparkSessionExtensions
  		#Configuring Catalog set to local
        .set('spark.sql.catalog.spark_catalog', 'org.apache.iceberg.spark.SparkSessionCatalog')
        .set('spark.sql.catalog.spark_catalog.type', 'hive')    
        .set('spark.sql.catalog.local', 'org.apache.iceberg.spark.SparkCatalog')
        .set('spark.sql.catalog.local.type', 'hadoop')
        .set('spark.sql.catalog.local.warehouse', '/home/docker/catalog')
)


spark = SparkSession.builder.config(conf=conf).getOrCreate()
print("Spark Running")

:: loading settings :: url = jar:file:/home/docker/.local/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/docker/.ivy2/cache
The jars for the packages stored in: /home/docker/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.3_2.12 added as a dependency
org.projectnessie.nessie-integrations#nessie-spark-extensions-3.3_2.12 added as a dependency
software.amazon.awssdk#bundle added as a dependency
software.amazon.awssdk#url-connection-client added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2d8df7af-d6e3-4503-bc5c-e0c4703b0ce5;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.3_2.12;1.3.1 in central
	found org.projectnessie.nessie-integrations#nessie-spark-extensions-3.3_2.12;0.67.0 in central
	found software.amazon.awssdk#bundle;2.17.178 in central
	found software.amazon.eventstream#eventstream;1.0.1 in central
	found software.amazon.awssdk#url-connection-client;2.17.178 in central
	found software.amazon.awssdk#utils;2.17.178 in central
	found org.reactivestreams#reactive-streams;1.0.3 in central

23/11/29 15:04:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Spark Running


## Experiment One
Read two years and update the table with Merge, if there is a collision we want to overwrite the column, but there are no collisions.

The expected result is that the final table have all columns from both years.

There is no schema change

In [3]:

## LOAD A CSV INTO AN SQL VIEW
csv_df = spark.read.format("csv").option("header", "true").load("../datasets/experiment1/df_open_2022_small.csv")
csv_df.createOrReplaceTempView("csv_open_2022")

## CREATE AN ICEBERG TABLE FROM THE SQL VIEW
spark.sql("CREATE TABLE IF NOT EXISTS local.db.experiment1 USING iceberg AS SELECT * FROM csv_open_2022")

## QUERY THE ICEBERG TABLE
spark.sql("SELECT count(*) FROM local.db.experiment1").show()
spark.sql("SELECT * FROM local.db.experiment1").show()

+--------+
|count(1)|
+--------+
|       9|
+--------+



+------------+---------------+---------+--------+------+------+-------------------+-------------------+--------+-------------+-----------+--------------------+---+------+------+-----------+------------+--------+----+
|competitorId| competitorName|firstName|lastName|status|gender|countryOfOriginCode|countryOfOriginName|regionId|   regionName|affiliateId|       affiliateName|age|height|weight|overallRank|overallScore|genderId|year|
+------------+---------------+---------+--------+------+------+-------------------+-------------------+--------+-------------+-----------+--------------------+---+------+------+-----------+------------+--------+----+
|      310970|  Saxon Panchik|    Saxon| Panchik|   ACT|     M|                 US|      United States|      31|North America|      19593|CrossFit East Nas...| 26| 69 in|180 lb|          1|          25|       1|2022|
|     1454081|    Matt Poulin|     Matt|  Poulin|   ACT|     M|                 US|      United States|      31|North America|      

In [8]:
# READ DATA FOR 2023
csv_df_2023 = spark.read.format("csv").option("header", "true").load("../datasets/experiment1/df_open_2023_small.csv")
csv_df_2023.createOrReplaceTempView("csv_open_2023")

# PERFORM MERGE ON THE TABLE
spark.sql("""
MERGE INTO local.db.experiment1 t USING (SELECT * FROM csv_open_2023) u ON t.competitorId = u.competitorId
WHEN MATCHED THEN UPDATE SET *
WHEN NOT MATCHED THEN INSERT *""")

# INSPECT THE TABLE AFTER
spark.sql("SELECT count(*) FROM local.db.experiment1").show()
spark.sql("SELECT * FROM local.db.experiment1").show()

+--------+
|count(1)|
+--------+
|      17|
+--------+



+------------+-----------------+---------+-----------+------+------+-------------------+-------------------+--------+------------------+-----------+--------------------+---+------+------+-----------+------------+--------+----+
|competitorId|   competitorName|firstName|   lastName|status|gender|countryOfOriginCode|countryOfOriginName|regionId|        regionName|affiliateId|       affiliateName|age|height|weight|overallRank|overallScore|genderId|year|
+------------+-----------------+---------+-----------+------+------+-------------------+-------------------+--------+------------------+-----------+--------------------+---+------+------+-----------+------------+--------+----+
|     1031875|  Roldan Goldbaum|   Roldan|   Goldbaum|   ACT|     M|                 MX|             Mexico|      34|North America West|      21264|Strong Suit CrossFit| 23|170 cm| 86 kg|          5|         526|       1|2023|
|     1454081|      Matt Poulin|     Matt|     Poulin|   ACT|     M|                 US|    

## Experiment TWO

The same as before, but 2023 has an additional column.

Lets see how Apache Iceberg handles the schema evolution

In [9]:
## LOAD A CSV INTO AN SQL VIEW
csv_df_e2_2022 = spark.read.format("csv").option("header", "true").load("../datasets/experiment2/df_open_2022_small.csv")
csv_df_e2_2022.createOrReplaceTempView("csv_open_2022_experiment_2")

## CREATE AN ICEBERG TABLE FROM THE SQL VIEW
spark.sql("CREATE TABLE IF NOT EXISTS local.db.experiment2 USING iceberg AS SELECT * FROM csv_open_2022_experiment_2")

## QUERY THE ICEBERG TABLE
spark.sql("SELECT count(*) FROM local.db.experiment2").show()
spark.sql("SELECT * FROM local.db.experiment2").show()
spark.sql("DESCRIBE local.db.experiment2").show()

+--------+
|count(1)|
+--------+
|       9|
+--------+

+------------+---------------+---------+--------+------+------+-------------------+-------------------+--------+-------------+-----------+--------------------+---+------+------+-----------+------------+--------+----+
|competitorId| competitorName|firstName|lastName|status|gender|countryOfOriginCode|countryOfOriginName|regionId|   regionName|affiliateId|       affiliateName|age|height|weight|overallRank|overallScore|genderId|year|
+------------+---------------+---------+--------+------+------+-------------------+-------------------+--------+-------------+-----------+--------------------+---+------+------+-----------+------------+--------+----+
|      310970|  Saxon Panchik|    Saxon| Panchik|   ACT|     M|                 US|      United States|      31|North America|      19593|CrossFit East Nas...| 26| 69 in|180 lb|          1|          25|       1|2022|
|     1454081|    Matt Poulin|     Matt|  Poulin|   ACT|     M|             

In [11]:
# READ DATA FOR 2023
csv_df_e2_2023 = spark.read.format("csv").option("header", "true").load("../datasets/experiment2/df_open_2023_small_new_col.csv")
csv_df_e2_2023.show()
csv_df_e2_2023.createOrReplaceTempView("csv_open_2023_experiment2")

# PERFORM MERGE ON THE TABLE
spark.sql("""
MERGE INTO local.db.experiment2 t USING (SELECT * FROM csv_open_2023_experiment2) u ON t.competitorId = u.competitorId
WHEN MATCHED THEN UPDATE SET *
WHEN NOT MATCHED THEN INSERT *""")

## QUERY THE ICEBERG TABLE
spark.sql("SELECT count(*) FROM local.db.experiment2").show()
spark.sql("SELECT * FROM local.db.experiment2").show()
spark.sql("DESCRIBE local.db.experiment2").show()

+------------+-----------------+---------+-----------+------+------+-------------------+-------------------+--------+------------------+-----------+--------------------+---+------+------+-----------+------------+--------+----+--------------------+
|competitorId|   competitorName|firstName|   lastName|status|gender|countryOfOriginCode|countryOfOriginName|regionId|        regionName|affiliateId|       affiliateName|age|height|weight|overallRank|overallScore|genderId|year|          aNewColumn|
+------------+-----------------+---------+-----------+------+------+-------------------+-------------------+--------+------------------+-----------+--------------------+---+------+------+-----------+------------+--------+----+--------------------+
|      469656|    Jeffrey Adler|  Jeffrey|      Adler|   ACT|     M|                 CA|             Canada|      35|North America East|      18059| CrossFit Wonderland| 29| 69 in|197 lb|          1|         107|       1|2023|thisIsANewColumn_...|
|      3

+--------+
|count(1)|
+--------+
|      17|
+--------+



+------------+-----------------+---------+-----------+------+------+-------------------+-------------------+--------+------------------+-----------+--------------------+---+------+------+-----------+------------+--------+----+
|competitorId|   competitorName|firstName|   lastName|status|gender|countryOfOriginCode|countryOfOriginName|regionId|        regionName|affiliateId|       affiliateName|age|height|weight|overallRank|overallScore|genderId|year|
+------------+-----------------+---------+-----------+------+------+-------------------+-------------------+--------+------------------+-----------+--------------------+---+------+------+-----------+------------+--------+----+
|     1031875|  Roldan Goldbaum|   Roldan|   Goldbaum|   ACT|     M|                 MX|             Mexico|      34|North America West|      21264|Strong Suit CrossFit| 23|170 cm| 86 kg|          5|         526|       1|2023|
|     1454081|      Matt Poulin|     Matt|     Poulin|   ACT|     M|                 US|    

The new column gets ignored. 

Lets try doing an alter in the table

In [12]:
spark.sql("""
ALTER TABLE local.db.experiment2
ADD COLUMNS (
    aNewColumn string comment 'new_column docs'
  )""")

# PERFORM MERGE ON THE TABLE
spark.sql("""
MERGE INTO local.db.experiment2 t USING (SELECT * FROM csv_open_2023_experiment2) u ON t.competitorId = u.competitorId
WHEN MATCHED THEN UPDATE SET *
WHEN NOT MATCHED THEN INSERT *""")

## QUERY THE ICEBERG TABLE
spark.sql("SELECT count(*) FROM local.db.experiment2").show()
spark.sql("SELECT * FROM local.db.experiment2").show()
spark.sql("DESCRIBE local.db.experiment2").show()

23/11/29 15:24:11 WARN BaseTransaction: Failed to load metadata for a committed snapshot, skipping clean-up


+--------+
|count(1)|
+--------+
|      17|
+--------+

+------------+-----------------+---------+-----------+------+------+-------------------+-------------------+--------+------------------+-----------+--------------------+---+------+------+-----------+------------+--------+----+--------------------+
|competitorId|   competitorName|firstName|   lastName|status|gender|countryOfOriginCode|countryOfOriginName|regionId|        regionName|affiliateId|       affiliateName|age|height|weight|overallRank|overallScore|genderId|year|          aNewColumn|
+------------+-----------------+---------+-----------+------+------+-------------------+-------------------+--------+------------------+-----------+--------------------+---+------+------+-----------+------------+--------+----+--------------------+
|     1031875|  Roldan Goldbaum|   Roldan|   Goldbaum|   ACT|     M|                 MX|             Mexico|      34|North America West|      21264|Strong Suit CrossFit| 23|170 cm| 86 kg|          5| 

Great, now it works! 

TODO: modify the type!

## Experiment THREE

The same as number 1, but 2023 has rows with collisions.

Lets see how Apache Iceberg handles collisions. Maybe replay

In [13]:
## LOAD A CSV INTO AN SQL VIEW
csv_df_e3_2022 = spark.read.format("csv").option("header", "true").load("../datasets/experiment3/df_open_2022_small.csv")
csv_df_e3_2022.createOrReplaceTempView("csv_open_2022_experiment_3")

## CREATE AN ICEBERG TABLE FROM THE SQL VIEW
spark.sql("CREATE TABLE IF NOT EXISTS local.db.experiment3 USING iceberg AS SELECT * FROM csv_open_2022_experiment_3")

## QUERY THE ICEBERG TABLE
spark.sql("SELECT count(*) FROM local.db.experiment3").show()
spark.sql("SELECT * FROM local.db.experiment3").show()
spark.sql("DESCRIBE local.db.experiment3").show()

+--------+
|count(1)|
+--------+
|       9|
+--------+

+------------+---------------+---------+--------+------+------+-------------------+-------------------+--------+-------------+-----------+--------------------+---+------+------+-----------+------------+--------+----+
|competitorId| competitorName|firstName|lastName|status|gender|countryOfOriginCode|countryOfOriginName|regionId|   regionName|affiliateId|       affiliateName|age|height|weight|overallRank|overallScore|genderId|year|
+------------+---------------+---------+--------+------+------+-------------------+-------------------+--------+-------------+-----------+--------------------+---+------+------+-----------+------------+--------+----+
|      310970|  Saxon Panchik|    Saxon| Panchik|   ACT|     M|                 US|      United States|      31|North America|      19593|CrossFit East Nas...| 26| 69 in|180 lb|          1|          25|       1|2022|
|     1454081|    Matt Poulin|     Matt|  Poulin|   ACT|     M|             

In [14]:
# READ DATA FOR 2023
csv_df_e3_2023 = spark.read.format("csv").option("header", "true").load("../datasets/experiment3/df_open_2023_small.csv")
csv_df_e3_2023.createOrReplaceTempView("csv_open_2023_experiment3")

# PERFORM MERGE ON THE TABLE
spark.sql("""
MERGE INTO local.db.experiment3 t USING (SELECT * FROM csv_open_2023_experiment3) u ON t.competitorId = u.competitorId
WHEN MATCHED THEN UPDATE SET *
WHEN NOT MATCHED THEN INSERT *""")

# INSPECT THE TABLE AFTER
spark.sql("SELECT count(*) FROM local.db.experiment3").show()
spark.sql("SELECT * FROM local.db.experiment3").show()

+--------+
|count(1)|
+--------+
|      17|
+--------+

+------------+--------------------+---------+-----------+------+------+-------------------+-------------------+--------+------------------+-----------+--------------------+---+------+------+-----------+------------+--------+----+
|competitorId|      competitorName|firstName|   lastName|status|gender|countryOfOriginCode|countryOfOriginName|regionId|        regionName|affiliateId|       affiliateName|age|height|weight|overallRank|overallScore|genderId|year|
+------------+--------------------+---------+-----------+------+------+-------------------+-------------------+--------+------------------+-----------+--------------------+---+------+------+-----------+------------+--------+----+
|     1031875|     Roldan Goldbaum|   Roldan|   Goldbaum|   ACT|     M|                 MX|             Mexico|      34|North America West|      21264|Strong Suit CrossFit| 23|170 cm| 86 kg|          5|         526|       1|2023|
|     1454081|         M

In [20]:
spark.sql("""
SELECT * FROM local.db.experiment3.history
   """).show()

+--------------------+-------------------+-------------------+-------------------+
|     made_current_at|        snapshot_id|          parent_id|is_current_ancestor|
+--------------------+-------------------+-------------------+-------------------+
|2023-11-29 15:31:...|6972414410548376259|               null|               true|
|2023-11-29 15:32:...|1468678298885254723|6972414410548376259|               true|
+--------------------+-------------------+-------------------+-------------------+



In [25]:
spark.sql("""
SELECT * FROM local.db.experiment3;
   """).show()

+------------+--------------------+---------+-----------+------+------+-------------------+-------------------+--------+------------------+-----------+--------------------+---+------+------+-----------+------------+--------+----+
|competitorId|      competitorName|firstName|   lastName|status|gender|countryOfOriginCode|countryOfOriginName|regionId|        regionName|affiliateId|       affiliateName|age|height|weight|overallRank|overallScore|genderId|year|
+------------+--------------------+---------+-----------+------+------+-------------------+-------------------+--------+------------------+-----------+--------------------+---+------+------+-----------+------------+--------+----+
|     1031875|     Roldan Goldbaum|   Roldan|   Goldbaum|   ACT|     M|                 MX|             Mexico|      34|North America West|      21264|Strong Suit CrossFit| 23|170 cm| 86 kg|          5|         526|       1|2023|
|     1454081|         Matt Poulin|     Matt|     Poulin|   ACT|     M|         

In [26]:
spark.sql("""
SELECT * FROM local.db.experiment3 VERSION AS OF 6972414410548376259;
   """).show()

+------------+---------------+---------+--------+------+------+-------------------+-------------------+--------+-------------+-----------+--------------------+---+------+------+-----------+------------+--------+----+
|competitorId| competitorName|firstName|lastName|status|gender|countryOfOriginCode|countryOfOriginName|regionId|   regionName|affiliateId|       affiliateName|age|height|weight|overallRank|overallScore|genderId|year|
+------------+---------------+---------+--------+------+------+-------------------+-------------------+--------+-------------+-----------+--------------------+---+------+------+-----------+------------+--------+----+
|      310970|  Saxon Panchik|    Saxon| Panchik|   ACT|     M|                 US|      United States|      31|North America|      19593|CrossFit East Nas...| 26| 69 in|180 lb|          1|          25|       1|2022|
|     1454081|    Matt Poulin|     Matt|  Poulin|   ACT|     M|                 US|      United States|      31|North America|      

## Experiment FOUR

The same as number 3, where we do some schema evolution by adding a column, but instead of altering the table, lets enable automatic schema evolution and see what happenes.


In [32]:
## LOAD A CSV INTO AN SQL VIEW
csv_df_e4_2022 = spark.read.format("csv").option("header", "true").load("../datasets/experiment2/df_open_2022_small.csv")
csv_df_e4_2022.createOrReplaceTempView("csv_open_2022_experiment_4")

## CREATE AN ICEBERG TABLE FROM THE SQL VIEW

spark.sql("CREATE TABLE IF NOT EXISTS local.db.experiment4 USING iceberg AS SELECT * FROM csv_open_2022_experiment_4")



## QUERY THE ICEBERG TABLE
spark.sql("SELECT count(*) FROM local.db.experiment4").show()
spark.sql("SELECT * FROM local.db.experiment4").show()
spark.sql("DESCRIBE local.db.experiment4").show()

+--------+
|count(1)|
+--------+
|       9|
+--------+



+------------+---------------+---------+--------+------+------+-------------------+-------------------+--------+-------------+-----------+--------------------+---+------+------+-----------+------------+--------+----+
|competitorId| competitorName|firstName|lastName|status|gender|countryOfOriginCode|countryOfOriginName|regionId|   regionName|affiliateId|       affiliateName|age|height|weight|overallRank|overallScore|genderId|year|
+------------+---------------+---------+--------+------+------+-------------------+-------------------+--------+-------------+-----------+--------------------+---+------+------+-----------+------------+--------+----+
|      310970|  Saxon Panchik|    Saxon| Panchik|   ACT|     M|                 US|      United States|      31|North America|      19593|CrossFit East Nas...| 26| 69 in|180 lb|          1|          25|       1|2022|
|     1454081|    Matt Poulin|     Matt|  Poulin|   ACT|     M|                 US|      United States|      31|North America|      

In [33]:
spark.sql("""ALTER TABLE local.db.experiment4 SET TBLPROPERTIES (
  'write.spark.accept-any-schema'='true'
)""")

23/11/29 16:01:48 WARN BaseTransaction: Failed to load metadata for a committed snapshot, skipping clean-up


DataFrame[]

In [35]:
# READ DATA FOR 2023
csv_df_e4_2023 = spark.read.format("csv").option("header", "true").load("../datasets/experiment2/df_open_2023_small_new_col.csv")
csv_df_e4_2023.show()

csv_df_e4_2023.writeTo("local.db.experiment4").option("mergeSchema","true").append()


## QUERY THE ICEBERG TABLE
spark.sql("SELECT count(*) FROM local.db.experiment4").show()
spark.sql("SELECT * FROM local.db.experiment4").show()

+------------+-----------------+---------+-----------+------+------+-------------------+-------------------+--------+------------------+-----------+--------------------+---+------+------+-----------+------------+--------+----+--------------------+
|competitorId|   competitorName|firstName|   lastName|status|gender|countryOfOriginCode|countryOfOriginName|regionId|        regionName|affiliateId|       affiliateName|age|height|weight|overallRank|overallScore|genderId|year|          aNewColumn|
+------------+-----------------+---------+-----------+------+------+-------------------+-------------------+--------+------------------+-----------+--------------------+---+------+------+-----------+------------+--------+----+--------------------+
|      469656|    Jeffrey Adler|  Jeffrey|      Adler|   ACT|     M|                 CA|             Canada|      35|North America East|      18059| CrossFit Wonderland| 29| 69 in|197 lb|          1|         107|       1|2023|thisIsANewColumn_...|
|      3

+--------+
|count(1)|
+--------+
|      18|
+--------+



+------------+-----------------+---------+-----------+------+------+-------------------+-------------------+--------+------------------+-----------+--------------------+---+------+------+-----------+------------+--------+----+--------------------+
|competitorId|   competitorName|firstName|   lastName|status|gender|countryOfOriginCode|countryOfOriginName|regionId|        regionName|affiliateId|       affiliateName|age|height|weight|overallRank|overallScore|genderId|year|          aNewColumn|
+------------+-----------------+---------+-----------+------+------+-------------------+-------------------+--------+------------------+-----------+--------------------+---+------+------+-----------+------------+--------+----+--------------------+
|      310970|    Saxon Panchik|    Saxon|    Panchik|   ACT|     M|                 US|      United States|      31|     North America|      19593|CrossFit East Nas...| 26| 69 in|180 lb|          1|          25|       1|2022|                null|
|     14

AnalysisException: Table or view not found: local.db.experiment; line 1 pos 9;
'DescribeRelation false, [col_name#6275, data_type#6276, comment#6277]
+- 'UnresolvedTableOrView [local, db, experiment], DESCRIBE TABLE, true


In [36]:
# READ DATA FOR 2023
csv_df_e4_2023 = spark.read.format("csv").option("header", "true").load("../datasets/experiment2/df_open_2023_small_new_col.csv")
csv_df_e4_2023.show()

csv_df_e4_2023.writeTo("local.db.experiment4").option("mergeSchema","true").append()


## QUERY THE ICEBERG TABLE
spark.sql("SELECT count(*) FROM local.db.experiment4").show()
spark.sql("SELECT * FROM local.db.experiment4").show()
spark.sql("DESCRIBE local.db.experiment4").show()

+------------+-----------------+---------+-----------+------+------+-------------------+-------------------+--------+------------------+-----------+--------------------+---+------+------+-----------+------------+--------+----+--------------------+
|competitorId|   competitorName|firstName|   lastName|status|gender|countryOfOriginCode|countryOfOriginName|regionId|        regionName|affiliateId|       affiliateName|age|height|weight|overallRank|overallScore|genderId|year|          aNewColumn|
+------------+-----------------+---------+-----------+------+------+-------------------+-------------------+--------+------------------+-----------+--------------------+---+------+------+-----------+------------+--------+----+--------------------+
|      469656|    Jeffrey Adler|  Jeffrey|      Adler|   ACT|     M|                 CA|             Canada|      35|North America East|      18059| CrossFit Wonderland| 29| 69 in|197 lb|          1|         107|       1|2023|thisIsANewColumn_...|
|      3

+--------+
|count(1)|
+--------+
|      27|
+--------+



+------------+-----------------+---------+-----------+------+------+-------------------+-------------------+--------+------------------+-----------+--------------------+---+------+------+-----------+------------+--------+----+--------------------+
|competitorId|   competitorName|firstName|   lastName|status|gender|countryOfOriginCode|countryOfOriginName|regionId|        regionName|affiliateId|       affiliateName|age|height|weight|overallRank|overallScore|genderId|year|          aNewColumn|
+------------+-----------------+---------+-----------+------+------+-------------------+-------------------+--------+------------------+-----------+--------------------+---+------+------+-----------+------------+--------+----+--------------------+
|      310970|    Saxon Panchik|    Saxon|    Panchik|   ACT|     M|                 US|      United States|      31|     North America|      19593|CrossFit East Nas...| 26| 69 in|180 lb|          1|          25|       1|2022|                null|
|     14